<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/C2-Jinja_and_Templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jinja Templates

We now discuss how to use _data_ within our _templates_. We will be using a template engine called `Jinja` which can be installed as follows:

In [ ]:
!pip install -U -q flask pyngrok

## Using Jinja for making HTML structure creation more flexible





### Saving pages to Google Drive

Let's also modify our code for the webserver. To avoid losing our templates moving forward, we will now store the templates under a folder in Google Drive.

In [ ]:
# This code connects Google Drive to Colab, and makes the
# contents of Google Drive available under "/content/drive/My Drive"
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create a folder on Google drive called "templates"
!mkdir -p /content/drive/My Drive/templates

In [ ]:
# @title Setup Code

import os
from flask import Flask, render_template
from pyngrok import ngrok
import datetime

os.environ["FLASK_DEBUG"] = "true"
# Open a ngrok tunnel to the HTTP server
ngrok_authtoken = '2WgDffgQcSJcesOPKNnZ1jvwxXJ_5sR4FFXtByxhjgkFB62QP'
ngrok.set_auth_token(ngrok_authtoken)
port = 5000
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1'")

### Using Templates

In our code so far, we have written a bit of HTML code within Python.

This is not really a great setup. It is better to separate Python code from HTML code.




We will save our HTML code under the `templates` directory that we created above. (It is under `/content/drive/My Drive/templates`.)

Let's start by creating our first template:

Create the follwing `index.html` and put it under the templates folder.

```html

<!DOCTYPE html>
<html>
<head>
    <title>My Flask App</title>
</head>
<body>
    <h1>Hello World!</h1>
    <h4>Welcome to the Flask Lesson</h4>
    <hr>
    <div style="color: green">
        <strong>Below an image is displayed</strong>
        <p>
        <img width="300" src="https://upload.wikimedia.org/wikipedia/en/thumb/5/58/NYU_logo.svg/1280px-NYU_logo.svg.png"/>
        <p>This is a simple example how to use HTML in Flask templates</p>
    </div>
    <p>More info you can find <a href="https://www.w3schools.com/html/default.asp">here</a></p>
</body>
</html>

```

In [ ]:
# Let's start and run our server:

# Notice that we add a pointer to our templates folder
app = Flask(__name__, template_folder = '/content/drive/My Drive/templates')
app.config["BASE_URL"] = public_url

# Notice the use of the render_template function
@app.route("/")
def home():
    return render_template("index.html")

public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1'")
app.run(use_reloader=False, port=port)

### Using Templates with variables


We will see that our templates will have a bit of a special structure. We will have HTML that can be automatically populated with data that we pass from Python.

Let's look at a simple example. Let's create an `index2.html` file in the templates folder. Notice in particular the `username`, and `today` elements:

```html
<html>
  <title>My main page</title>
  <body>
    <h1>Hello, {{ username }}!</h1>
    <h1>Today is {{ today }}</h1>
  </body>
</html>
```




Under the covers, the `render_template` function invokes the [Jinja2](http://jinja.pocoo.org/) templating engine that is part of the Flask framework. Jinja2 substitutes `{{ ... }}` blocks with the corresponding values provided as template arguments.

In [ ]:
# Let's start and run our server:

# Notice that we add a pointer to our templates folder
app = Flask(__name__, template_folder = '/content/drive/My Drive/templates')
app.config["BASE_URL"] = public_url

# Notice the use of the render_template function
# and the use of the parameters username and today
# that we pass to the HTML file index.html
@app.route("/")
def home():
    today = datetime.date.today()
    return render_template("index2.html", username="Panos Ipeirotis", today=today)

public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1'")
app.run(use_reloader=False, port=port)

As you may see both keys (`username` and `today`) written in `{{ }}` brackets were replaced by their values defined in the `render_template` function (`"Panos Ipeirotis"` and respective today's date).


## Jinja iterators

The Jinja2 templates also support control statements (`if - else`) and `for`-loops, given inside `{% ... %}` blocks.

For example, here is a `for` loop, that displays a list of persons name and their age provided in the varaible `persons` ([HTML tags `<ul>` and  `<li>`](https://www.w3schools.com/tags/tag_li.asp) are using for lists creation in HTML structure):

Modify the usual web server code, and add the following function:

```python
@app.route("/actors")
def actors():
    persons = [
        {"name": "Robert De Niro", "age": 73},
        {"name": "Leonardo DiCaprio", "age": 42},
        {"name": "Tom Hanks", "age": 60},
        {"name": "Tom Cruise", "age": 54}
    ]
    return render_template('actors.html', title="My favorite actors", actors=persons)
```

And create a file in "actors.html" in under the `templates` folder.

```html
<body>
    <h2>{{ title }}</h2>
    <ul>
        {% for i in actors %}
        <li>Name: {{ i.name }}; age: {{ i.age }}</li>
        {% endfor %}
    </ul>
</body>
```

The logic is very similar to Python's one, but pay attention that `for` loop must be closed by `endfor` clause. <br/>


In [ ]:
# Let's start and run our server
# Remember to create the actors.html file under templates

app = Flask(__name__, template_folder = '/content/drive/My Drive/templates')
app.config["BASE_URL"] = public_url

@app.route("/actors")
def actors():
    persons = [
        {"name": "Robert De Niro", "age": 73},
        {"name": "Leonardo DiCaprio", "age": 42},
        {"name": "Tom Hanks", "age": 60},
        {"name": "Tom Cruise", "age": 54}
    ]
    return render_template('actors.html', title="My favorite actors", actors=persons)

public_url = ngrok.connect(port).public_url
print(f" * Our page is at {public_url}/actors")
app.run(use_reloader=False, port=port)

### Conditional statements

Let's demonstrate how to use `if`-statements and in the above template make bold all rows where actor is elder than 50 and underline actors who is 60 years old.

```html
<body>
    <h2>{{ title }}</h2>
    <ul>
        {% for i in actors %}
            <li>
                {% if i.age == 60 %}
                    <u>Name: {{ i.name }}; age: {{ i.age }}</u>
                {% elif i.age > 50 %}
                    <strong>Name: {{ i.name }}; age: {{ i.age }}</strong>
                {% else %}
                    Name: {{ i.name }}; age: {{ i.age }}
                {% endif %}
            </li>
        {% endfor %}
    </ul>
</body>
```

Also `if`-statement should be closed by `endif` command.

You can

The capabilities proposing by Jinja2 usage are more wide than we have shown, but it will be absolutely enough for majority of templates creation. More information you may find [here](http://jinja.pocoo.org/docs/2.9/templates/).

### Exercise

There is a list with food name and calories

>```python
food = [
    {"food": "Apple pie, 1 piece", "calories": 405, "protein": 3, "fat": 18, "carbohydrate": 60},
    {"food": "Banana", "calories": 105, "protein": 1, "fat": 0, "carbohydrate": 27},  
    {"food": "Beaf steak", "calories": 240, "protein": 23, "fat": 15, "carbohydrate": 0},
    {"food": "Carrot", "calories": 30, "protein": 1, "fat": 0, "carbohydrate": 7},
    {"food": "Chicken soup, 1 cup", "calories": 60, "protein": 4, "fat": 2, "carbohydrate": 7}
]
```

Using ONLY Jinja2 template syntax display at the new page http://your_ip:5000/food the list of foods. Feel free to add conditional formatting, eg  when the number of calories exceeds a limit.

**Hint:** You need to create a new HTML template and a new view function the webserver code and set fot it the respective URL.